# 機器學習常犯錯的十件事

- 資料面
  - 資料收集與處理不當
  - 訓練集與測試集的類別分佈不一致
  - 沒有資料視覺化的習慣
  - 使用 LabelEcoder 為特徵編碼
  - 資料處理不當導致資料洩漏


- 模型面
  - 僅使用測試集評估模型好壞
  - 在沒有交叉驗證的情況下判斷模型性能
  - 分類問題僅使用準確率作為衡量模型的指標
  - 迴歸問題僅使用 R2 分數評估模型好壞
  - 任何事情別急著想用 AI 解決

> 詳細內容請[參閱](https://ithelp.ithome.com.tw/articles/10279778)

## 2. 訓練集與測試集的類別分佈不一致
在分類的資料中，初學者常見的錯誤是忘記使用分層抽樣 (stratify) 來對訓練集和測試集進行切割。當測試集的分佈盡可能與訓練相同情況下，模型才更有可能得到更準確的預測。然而在分類的問題中，我們必須更關注每個類別的資料分佈比例。以下舉個例子：假設我們有三個標籤的類別，而這三個類別的分佈比例分別為 4:3:3。同理我們在進行資料切割的時候必須確保訓練集與測試集需要有相同的資料分佈比例。

大家應該都使用過 Sklearn 的 `train_test_split` 進行資料切割。在此方法中 Sklearn 提供了一個 `stratify` 參數達到分層隨機抽樣的目的。特別是在原始數據中樣本標籤分佈不均衡時非常有用，一些分類問題可能會在目標類的分佈中表現出很大的不平衡：例如，負樣本與正樣本比例懸殊(信用卡盜刷預測、離職員工預測)。以下用紅酒分類預測來進行示範，首先我們不使用 `stratify` 隨機切割資料並查看資料切割前後的三種類別比例。

In [ ]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X, y = load_wine(return_X_y=True)

# 查看全部資料三種類別比例
pd.Series(y).value_counts(normalize=True)

1    0.398876
0    0.331461
2    0.269663
dtype: float64

In [ ]:
# 實驗一: 不使用 stratify 進行切割資料
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25)

# 查看訓練集三種類別比例
print('訓練集')
print(pd.Series(y_train).value_counts(normalize=True))
# 查看測試集三種類別比例
print('測試集')
print(pd.Series(y_test).value_counts(normalize=True))

訓練集
1    0.386364
0    0.363636
2    0.250000
dtype: float64
測試集
1    0.402985
0    0.320896
2    0.276119
dtype: float64


從上面切出來的訓練集與測試集可以發現三個類別的資料分佈比例都不同。因此我們可以使用 `stratify` 參數再切割一次。我們可以發現將 `stratify` 設置為目標 (y) 在訓練和測試集中產生相同的分佈。因為改變的類別的比例是一個嚴重的問題，可能會使模型更偏向於特定的類別。因此訓練資料的分佈必須要與實際情況越接近越好。

In [ ]:
# 實驗二: 使用 stratify 進行切割資料
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25, stratify=y)

# 查看訓練集三種類別比例
print('訓練集')
print(pd.Series(y_train).value_counts(normalize=True))
# 查看測試集三種類別比例
print('測試集')
print(pd.Series(y_test).value_counts(normalize=True))

訓練集
1    0.386364
0    0.340909
2    0.272727
dtype: float64
測試集
1    0.402985
0    0.328358
2    0.268657
dtype: float64


## 4. 使用 LabelEncoder 為特徵編碼
通常我們要為類別的特徵進行編碼，直覺會想到 Sklearn 的 [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)。但是如果一個資料集中有多個特徵是屬於類別型的資料，豈不是很麻煩?必須要一個一個呼叫 LabelEncoder 分別為這些特徵進行轉換。如果你看到這邊有同感的，在這裡要告訴你事實並非如此！我們看看 在官方文件下 LabelEncoder 的描述：

> This transformer should be used to encode target values, i.e. y, and not the input X.

簡單來說 LabelEncoder 只是被用來編碼輸出項 y 而已的！你還在用它來編碼你的每個 x 嗎？（暈

那麼我們該用什麼方法來編碼有順序的類別特徵呢？如果你仔細閱讀有關編碼分類特徵的 Sklearn 用戶指南，你會看到它清楚地說明：

> To convert categorical features to integer codes, we can use the OrdinalEncoder. This estimator transforms each categorical feature to one new feature of integers (0 to n_categories - 1)

看到這邊大家應該知道閱讀官方文件的重要性吧！官方文件中建議 x 項的輸入特徵可以採用 [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) 一次為所有特徵依序做 Label Encoding。OrdinalEncoder 編碼器的使用方式如下：

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)

print(enc.categories_)
# 測試編碼
enc.transform([['Female', 3], ['Male', 1]])

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]


array([[0., 2.],
       [1., 0.]])

以上的範例是 X 有三筆資料，每筆資料都有兩個特徵。我們可以發現第一個特徵是性別 Male 與 Female，因此 OrdinalEncoder 會依造字母開頭做排序 Female 編碼為 0 而 Male 編碼為 1。另外第二個特徵為數字 1、2、3，同理依序為他們編碼成 0、1、2。只需閱讀官方文檔和用戶指南，你就可以了解很多關於 Sklearn 的知識！是不是很棒～

> 可以自行調整順序 categories=[['Male','Female'],[3,2,1]]

## 5. 資料處理不當導致資料洩漏
最簡單的解決辦法，就是不要使用 `fit()` 一次轉換所有的資料。在做任何資料轉換之前要先確保訓練集與測試集已經完整地被切開。即使切開後也不要再拿測試集呼叫 `fit()` 或 `fit_transform()`，這一樣會導致相同問題發生。因為訓練集和測試集必須進行相同的轉換，依照官方的範例我們必須先使用 `fit_transform()` 在訓練集上進行擬合與轉換。這確保了轉換器僅從訓練集學習，從中找出參數例如平均值與變異數並同時對其進行變換。接著使用 `transform()` 方法在測試資料上進行轉換，根據從訓練數據中學到的訊息進行轉換。


### 不建議方法

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X, y = load_iris(return_X_y=True)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=44)

In [ ]:
print('\nStandardScaler 縮放過後訓練集的最小值 : ', X_scaled.min(axis=0))
print('StandardScaler 縮放過後訓練集的最大值 : ', X_scaled.max(axis=0))


StandardScaler 縮放過後訓練集的最小值 :  [0. 0. 0. 0.]
StandardScaler 縮放過後訓練集的最大值 :  [1. 1. 1. 1.]


![](https://i.imgur.com/9AhNggd.png)

### 建議方法

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=44)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print('\nStandardScaler 縮放過後訓練集的最小值 : ', X_train_scaled.min(axis=0))
print('StandardScaler 縮放過後訓練集的最大值 : ', X_train_scaled.max(axis=0))


StandardScaler 縮放過後訓練集的最小值 :  [0. 0. 0. 0.]
StandardScaler 縮放過後訓練集的最大值 :  [1. 1. 1. 1.]


In [ ]:
print('\nStandardScaler 縮放過後測試集的最小值 : ', X_test_scaled.min(axis=0))
print('StandardScaler 縮放過後測試集的最大值 : ', X_test_scaled.max(axis=0))


StandardScaler 縮放過後測試集的最小值 :  [ 0.02777778 -0.09090909 -0.01785714  0.        ]
StandardScaler 縮放過後測試集的最大值 :  [0.94444444 0.86363636 1.03571429 0.95833333]


![](https://i.imgur.com/GXoqZg2.png)

## 8. 分類問題僅使用準確率作為衡量模型的指標
對於多元類分類的問題更是應該注意你的模型評估指標。如果達到 80% 的準確率，是否意味著模型在預測類別1、類別2、類別3甚至所有類時一樣準確呢？一般的準確率永遠無法回答此類問題，但幸運的是其他分類指標提供了更多的訊息指標。它就是[混淆矩陣](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)(confusion matrix)。

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
y_true = [2, 0, 2, 2, 0, 1, 1]
y_pred = [0, 0, 2, 2, 0, 2, 1]
print(confusion_matrix(y_true, y_pred))

[[2 0 0]
 [0 1 1]
 [1 0 2]]


In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       1.00      0.50      0.67         2
           2       0.67      0.67      0.67         3

    accuracy                           0.71         7
   macro avg       0.78      0.72      0.71         7
weighted avg       0.76      0.71      0.70         7

